# Chainlit Help - ÌîÑÎ°¨ÌîÑÌä∏ ÌÜ†ÌÅ∞ Î∂ÑÏÑù

ÌòÑÏû¨ ÌîÑÎ°úÏ†ùÌä∏ÏóêÏÑú ÏÇ¨Ïö©ÌïòÎäî ÌîÑÎ°¨ÌîÑÌä∏Ïùò ÌÜ†ÌÅ∞ ÏàòÎ•º Í≥ÑÏÇ∞Ìï©ÎãàÎã§.

Ï∞∏Í≥†: [Gemini API ÌÜ†ÌÅ∞ Í≥ÑÏÇ∞ Î¨∏ÏÑú](https://ai.google.dev/gemini-api/docs/tokens?authuser=1&hl=ko&lang=python)


In [18]:
import os
import json
from google import genai
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate

# ÌôòÍ≤ΩÎ≥ÄÏàò Î°úÎìú
load_dotenv('app/.env')

# Google AI API ÌÇ§ ÏÑ§Ï†ï
api_key = os.getenv('GOOGLE_API_KEY')
if not api_key:
    print("GOOGLE_API_KEYÍ∞Ä ÏÑ§Ï†ïÎêòÏßÄ ÏïäÏïòÏäµÎãàÎã§. app/.env ÌååÏùºÏùÑ ÌôïÏù∏Ìï¥Ï£ºÏÑ∏Ïöî.")
else:
    client = genai.Client(api_key=api_key)
    print("Google AI ÌÅ¥ÎùºÏù¥Ïñ∏Ìä∏ Ï¥àÍ∏∞Ìôî ÏôÑÎ£å")


Google AI ÌÅ¥ÎùºÏù¥Ïñ∏Ìä∏ Ï¥àÍ∏∞Ìôî ÏôÑÎ£å


In [19]:
# Ïª®ÌÖçÏä§Ìä∏ ÌååÏùºÎì§ Î°úÎìú
def load_context_files():
    context_files = {
        'codebase': 'app/context/codebase.txt',
        'documentation': 'app/context/documentation.txt', 
        'cookbook': 'app/context/cookbook.txt'
    }
    
    context_content = {}
    
    for name, path in context_files.items():
        try:
            with open(path, 'r', encoding='utf-8') as f:
                content = f.read()
                context_content[name] = content
                print(f"{name}: {len(content):,} Î¨∏Ïûê")
        except FileNotFoundError:
            print(f"‚ö†Ô∏è {path} ÌååÏùºÏùÑ Ï∞æÏùÑ Ïàò ÏóÜÏäµÎãàÎã§.")
            context_content[name] = ""
    
    return context_content

context_content = load_context_files()


codebase: 62,648 Î¨∏Ïûê
documentation: 257,654 Î¨∏Ïûê
cookbook: 44,054 Î¨∏Ïûê


In [31]:
# RAG ÌîÑÎ°¨ÌîÑÌä∏ ÌÖúÌîåÎ¶ø Î°úÎìú
with open('app/prompts/rag.json', 'r', encoding='utf-8') as f:
    rag_prompt = json.load(f)

print("RAG ÌîÑÎ°¨ÌîÑÌä∏ ÏÑ§Ï†ï:")
print(f"Î™®Îç∏: {rag_prompt['settings']['model']}")
print(f"Ïò®ÎèÑ: {rag_prompt['settings']['temperature']}")
print("\nÏãúÏä§ÌÖú ÌîÑÎ°¨ÌîÑÌä∏ ÌÖúÌîåÎ¶ø:")
print(rag_prompt['template_messages'][0]['content'][:200] + "...")


RAG ÌîÑÎ°¨ÌîÑÌä∏ ÏÑ§Ï†ï:
Î™®Îç∏: gemini-2.0-flash
Ïò®ÎèÑ: 0.2

ÏãúÏä§ÌÖú ÌîÑÎ°¨ÌîÑÌä∏ ÌÖúÌîåÎ¶ø:
You are a helpful assistant who answers questions related to Chainlit in valid markdown. Keep it short, and if available prefer responding with code. For documentation links, always use fully formed u...


In [32]:
langchain_template = rag_prompt['template_messages'][0]['content'].replace('{{', '{').replace('}}', '}')

# Ïã§Ï†ú ÏãúÏä§ÌÖú ÌîÑÎ°¨ÌîÑÌä∏ Íµ¨ÏÑ± (app.pyÏôÄ ÎèôÏùºÌïòÍ≤å)
system_prompt_template = PromptTemplate(
    input_variables=["codebase", "documentation", "cookbook"],
    template=langchain_template
)

In [33]:
system_prompt_template

PromptTemplate(input_variables=['codebase', 'cookbook', 'documentation'], input_types={}, partial_variables={}, template='You are a helpful assistant who answers questions related to Chainlit in valid markdown. Keep it short, and if available prefer responding with code. For documentation links, always use fully formed urls using this domain: `https://docs.chainlit.io`. Do NOT create APIs, methods or anything else outside of provided context. NEVER hallucinate or extrapolate on the context.\n\nContext:\n<codebase>\n{codebase}\n</codebase>\n<documentation>\n{documentation}\n</documentation>\n<cookbook>\n{cookbook}\n</cookbook>\n\n')

In [34]:
# ÌÖúÌîåÎ¶øÏóê Ïª®ÌÖçÏä§Ìä∏ ÎÇ¥Ïö© ÏÇΩÏûÖ
formatted_system_prompt = system_prompt_template.format_prompt(
    codebase=context_content['codebase'],
    documentation=context_content['documentation'],
    cookbook=context_content['cookbook']
)

In [35]:
formatted_system_prompt

StringPromptValue(text='You are a helpful assistant who answers questions related to Chainlit in valid markdown. Keep it short, and if available prefer responding with code. For documentation links, always use fully formed urls using this domain: `https://docs.chainlit.io`. Do NOT create APIs, methods or anything else outside of provided context. NEVER hallucinate or extrapolate on the context.\n\nContext:\n<codebase>\nDirectory structure:\n‚îî‚îÄ‚îÄ chainlit-chainlit/\n    ‚îî‚îÄ‚îÄ libs/\n        ‚îî‚îÄ‚îÄ react-client/\n            ‚îî‚îÄ‚îÄ src/\n                ‚îú‚îÄ‚îÄ context.ts\n                ‚îú‚îÄ‚îÄ index.ts\n                ‚îú‚îÄ‚îÄ state.ts\n                ‚îú‚îÄ‚îÄ useAudio.ts\n                ‚îú‚îÄ‚îÄ useChatData.ts\n                ‚îú‚îÄ‚îÄ useChatInteract.ts\n                ‚îú‚îÄ‚îÄ useChatMessages.ts\n                ‚îú‚îÄ‚îÄ useChatSession.ts\n                ‚îú‚îÄ‚îÄ useConfig.ts\n                ‚îú‚îÄ‚îÄ api/\n                ‚îÇ   ‚îú‚îÄ‚îÄ index.tsx\n

In [37]:
print(f"ÏôÑÏÑ±Îêú ÏãúÏä§ÌÖú ÌîÑÎ°¨ÌîÑÌä∏ Í∏∏Ïù¥: {len(formatted_system_prompt.text):,} Î¨∏Ïûê")

ÏôÑÏÑ±Îêú ÏãúÏä§ÌÖú ÌîÑÎ°¨ÌîÑÌä∏ Í∏∏Ïù¥: 364,823 Î¨∏Ïûê


In [40]:
# Ïª®ÌÖçÏä§Ìä∏ ÌÜ†ÌÅ∞ Ïàò Í≥ÑÏÇ∞
if api_key:
    model_name = rag_prompt['settings']['model']  # gemini-2.0-flash
    
    print(f"Î™®Îç∏: {model_name}")
    print("=" * 50)
    
    # Ï†ÑÏ≤¥ ÏãúÏä§ÌÖú ÌîÑÎ°¨ÌîÑÌä∏
    total_tokens = client.models.count_tokens(model=model_name, contents=[formatted_system_prompt.text])
    print(f"\n Ï†ÑÏ≤¥ ÏãúÏä§ÌÖú ÌîÑÎ°¨ÌîÑÌä∏: {total_tokens.total_tokens:,} ÌÜ†ÌÅ∞")
    
    print("\n" + "=" * 50)
    print("ÌÜ†ÌÅ∞ Î∂ÑÏÑù ÏöîÏïΩ:")
    print(f"- Ï¥ùÌï©: {total_tokens.total_tokens:,} ÌÜ†ÌÅ∞")
else:
    print("API ÌÇ§Í∞Ä ÏóÜÏñ¥ÏÑú ÌÜ†ÌÅ∞ Í≥ÑÏÇ∞ÏùÑ ÏàòÌñâÌï† Ïàò ÏóÜÏäµÎãàÎã§.")


Î™®Îç∏: gemini-2.0-flash

 Ï†ÑÏ≤¥ ÏãúÏä§ÌÖú ÌîÑÎ°¨ÌîÑÌä∏: 100,081 ÌÜ†ÌÅ∞

ÌÜ†ÌÅ∞ Î∂ÑÏÑù ÏöîÏïΩ:
- Ï¥ùÌï©: 100,081 ÌÜ†ÌÅ∞
